# Notebook de la dimensión Tipo Novedad - Proceso ETL

## 1. Configuración de conexiones (fuente y bodega)

In [1]:
import yaml
from sqlalchemy import create_engine
import pandas as pd

# Leer configuración desde archivo externo
with open('config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_fuente = config['fuente']
    config_bodega = config['bodega']

# URLs de conexión
url_fuente = (f"{config_fuente['driver']}://{config_fuente['user']}:{config_fuente['password']}@"
              f"{config_fuente['host']}:{config_fuente['port']}/{config_fuente['db']}")

url_bodega = (f"{config_bodega['driver']}://{config_bodega['user']}:{config_bodega['password']}@"
              f"{config_bodega['host']}:{config_bodega['port']}/{config_bodega['db']}")

# Engines SQLAlchemy
engine_fuente = create_engine(url_fuente)
engine_bodega = create_engine(url_bodega)

## 2. Extracción de datos desde la base de datos - Extract

In [2]:
from sqlalchemy import inspect

# Saber las columnas que hay en la base de datos OLTP
inspector = inspect(engine_fuente)
tablas = inspector.get_table_names()
print("Tablas de la base de datos:")
print(tablas)

Tablas de la base de datos:
['admin_interface_theme', 'areas_cliente', 'auth_user_user_permissions', 'clientes_coordinador', 'clientes_datosmensajero', 'clientes_mensajeroaquitoy_clientes', 'auth_permission', 'auth_group', 'auth_user_groups', 'auth_user', 'authtoken_token', 'ciudad', 'cliente', 'clientes_mensajeroaquitoy', 'area', 'django_migrations', 'departamento', 'clientes_usuarioaquitoy', 'django_admin_log', 'django_session', 'mensajeria_novedadesservicio', 'mensajeria_tiponovedad', 'mensajeria_origenservicio', 'mensajeria_tipopago', 'mensajeria_estado', 'mensajeria_tiposervicio', 'mensajeria_servicio', 'mensajeria_estadosservicio', 'mensajeria_tipovehiculo', 'sede', 'auth_group_permissions', 'django_content_type', 'tipo_cliente', 'mensajeria_destinoservicio', 'mensajeria_documentoasociado']


In [4]:
# Consulta SQL
query = """
SELECT 
    id AS id_tipo_novedad,
    nombre AS descripcion_novedad
FROM mensajeria_tiponovedad;
"""

# Ejecutar consulta
df_tipo_novedad = pd.read_sql(query, engine_fuente)
df_tipo_novedad

,id_tipo_novedad,descripcion_novedad
0,2,No puedo continuar
1,1,Novedades del servicio


In [5]:
from datetime import datetime

# Agregar columna de fecha de carga
df_tipo_novedad["fecha_carga"] = datetime.now().date()

## 4. Guardado de la dimensión - Load

In [6]:
# Guardar como archivo CSV
df_tipo_novedad.to_csv("dim_tipo_novedad.csv", index=False)

In [7]:
# Cargar en la bodega de datos OLAP
df_tipo_novedad.to_sql("dim_tipo_novedad", engine_bodega, if_exists="replace", index=False)

2

In [8]:
# Verificación del archivo
df_verificacion = pd.read_csv("dim_tipo_novedad.csv")
print(df_verificacion.head())

   id_tipo_novedad     descripcion_novedad fecha_carga
0                2      No puedo continuar  2025-06-05
1                1  Novedades del servicio  2025-06-05
